# Deep Learning 第10回:
# 　　　　　　パラメータの自動最適化(過学習編)

## 今回の目標
前回に引き続きパラメータの自動最適化  

前回は以下の最適化を行った
- パラメータ更新
- 重みの初期値

今回は特に過学習を抑制するためのパラメータ調整を紹介
- Batch Normalization
- 正則化
- ハイパーパラメータの検証

## Bach Normalization
各層でのアクティベーション分布を正規化よって、適度な広がりを持つようにする  
- 学習を速く進行させることができる(学習係数を大きくすることができる)
- 初期値にそれほど依存しない(初期値に対してそこまで神経質にならなくてよい)
- 過学習を抑制する

Batch Normのレイヤを入れる
![](https://tanaka-tom.github.io/update2/bach-norm-layer.svg)

$ { \mu  }_{ B }\leftarrow \frac { 1 }{ m } \sum _{ i=1 }^{ m }{ { x }_{ i } } $

$ { { \sigma  }_{ B }^{ 2 } }\leftarrow \frac { 1 }{ m } \sum _{ i=1 }^{ m }{ { (x }_{ i } } -{ \mu  }_{ B })^{ 2 } $

${ \hat { x }  }_{ i }\leftarrow \frac { { x }_{ i }-{ \mu  }_{ B } }{ \sqrt { { \sigma  }_{ B }^{ 2 }+\varepsilon  }  } $

- $\varepsilon$ : 0徐算を防止のための小さい値(10e-7)
- ${ \mu  }_{ B }$ : 平均
- ${ { \sigma  }_{ B } }$ : 分散

ミニバッチの入力データ $\left\{ { x }_{ 1 },{ x }_{ 2 },\cdots ,{ x }_{ m } \right\} $ を  
平均 0、分散 1 のデータ $ \left\{ { \hat { x } }_{ 1 },{ \hat { x } }_{ 2 },\cdots ,{ \hat { x } }_{ m } \right\} $  に変換する  
この処理を活性化関数の前(もしくは後)に挿入し、データの分布の偏りを減らす

さらに Bach Normレイヤでは、正規化されたデータに対して固有のスケールとシフトで変換を行う

${ y }_{ i }\leftarrow \gamma { \hat { x }  }_{ i }+\beta $

$\gamma = 1,\beta = 0$ からスタートして、学習によって適した値に調整される

## 正則化
過学習抑制

### 過学習の原因
- パラメータを大量に持ち、表現力の高いモデルであること
- 訓練データが少ないこと

訓練データの正解率が100%付近に到達しても、テストデータの正解率が上昇しない場合には過学習の可能性が高い

### Weight Decay
大きな重みにペナルティを与える方法

### Dropout
ニューロンをランダムで削除する方法

![](https://tanaka-tom.github.io/update2/dropout.png)
参照：　http://cs231n.github.io/neural-networks-2/

In [1]:
class Dropout:
	def __init__(self, dropout_ratio=0.5):
		self.dropout_ratio = dropout_ratio
		self.mask = None

	def forward(self, x, train_flg=True):
		if train_flg:
			self.mask = np.random.rand(*x.shape) > self.dropout_ratio
			return x * self.mask
		else:
			return x * (1.0 - self.dropout_ratio)

	def backword(self, dout):
		return dout * self.mask

## ハイパーパラメータ検証

### ステップ0
ハイパーパラメータの範囲を設定する。
### ステップ1
設定されたハイパーパラメータの範囲から、ランダムにサンプリングする。
### ステップ2
ステップ1でサンプリングされたハイパーパラメータの値を使用して学習を行 い、検証データで認識精度を評価する(ただし、エポックは小さく設定)。
### ステップ3
ステップ1とステップ2をある回数(100 回など)繰り返し、それらの認識精 度の結果から、ハイパーパラメータの範囲を狭める。
